# Library 설치

In [230]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import time, random
import pandas as pd
import json
import numpy as np
import re
from tqdm import tqdm

## 옵션 설정

In [ ]:
# 소수점 아래 2자리까지 표시하도록 설정
pd.options.display.float_format = '{:.2f}'.format

# ... 드라이버 설정 부분 ...
chrome_options = Options()
# 실제 브라우저처럼 보이게 하는 User-Agent 설정
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
chrome_options.add_argument(f"user-agent={user_agent}")

# 봇으로 감지될 가능성을 줄이는 추가 옵션들
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
except Exception as e:
    print(f"드라이버 초기화 중 오류 발생: {e}")
    exit()

In [ ]:
# --- 2. 수동 로그인 및 검색 페이지 이동 ---
try:
    # 먼저 네이버 메인 페이지로 이동하여 로그인
    driver.get("https://www.naver.com")
    input("▶ 브라우저에서 네이버 로그인을 완료한 후, 터미널로 돌아와 Enter 키를 눌러주세요...")

    # 실제 검색을 원하는 페이지로 이동
    search_url = "https://search.shopping.naver.com/search/all?query=탈모샴푸" # 예시 URL
    driver.get(search_url)
    print("검색 페이지로 이동했습니다. 스크롤을 시작합니다.")
    time.sleep(random.uniform(2, 3))

except Exception as e:
    print(f"로그인 또는 페이지 이동 중 오류: {e}")
    driver.quit()
    exit()

In [ ]:
# --- 1. 초기 크롤링 ---
# 최종 데이터를 담을 리스트
link_list, title_list, store_list, original_price_list, current_price_list, discount_rate_list = [], [], [], [], [], []

# 이미 수집한 상품 링크를 저장하여 중복을 방지하는 set
processed_links = set()

# 상품 카드를 찾는 CSS 선택자
card_selector = 'div#composite-card-list ul.compositeCardList_product_list__Ih4JR li.compositeCardContainer_composite_card_container__jr8cb'

# 연속으로 새 상품을 찾지 못한 스크롤 횟수
empty_scroll_count = 0 
# 최대 허용 연속 실패 횟수
max_empty_scrolls = 3

# 메인 루프 시작
while True:
    # 이번 수집 주기에서 새로운 상품을 찾았는지 기록하는 플래그
    found_new_product_in_cycle = False
    
    # 현재 화면의 HTML을 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    current_cards = soup.select(card_selector)

    # 현재 보이는 상품들을 순회하며 새로운 상품만 골라내어 수집
    for card in current_cards:
        try:
            link_tag = card.select_one('a.basicProductCard_link__urzND')
            # 링크 태그가 없는 카드는 건너뛰기 (광고 등 비상품 요소)
            if not link_tag:
                continue
                
            link = link_tag['href']
            
            # 이전에 수집한 적 없는 새로운 상품인 경우에만 데이터 추출
            if link not in processed_links:
                found_new_product_in_cycle = True # ✨ 새로운 상품 발견! 플래그를 True로 변경
                processed_links.add(link)
                
                title = card.select_one('strong.productCardTitle_product_card_title__eQupA').text.strip()
                store = card.select_one('span.productCardMallLink_mall_name__5oWPw').text.strip()
                
                original_price_tag = card.select_one('span.priceTag_original_price__jyZRY')
                original_price = int(original_price_tag.text.replace('할인 전 판매가', '').replace('원', '').replace(',', '').strip()) if original_price_tag else np.nan

                current_price_tag = card.select_one('span.priceTag_number__1QW0R')
                current_price = int(current_price_tag.text.replace(',', '').strip()) if current_price_tag else np.nan
                
                discount_tag = card.select_one('span.priceTag_discount__F_ZXz')
                discount_rate = int(discount_tag.text.replace('할인', '').replace('%', '').strip()) / 100 if discount_tag else np.nan

                link_list.append(link)
                title_list.append(title)
                store_list.append(store)
                original_price_list.append(original_price)
                current_price_list.append(current_price)
                discount_rate_list.append(discount_rate)
                
                print(f"✅ [{len(processed_links)}] 수집 성공: {title}")
        except Exception as e:
            # 개별 카드에서 오류 발생 시 (데이터 구조가 다른 경우 등) 건너뛰기
            print(f"⚠️ 개별 상품 처리 중 오류. 건너뜁니다. ({e})")
            
    # --- 스크롤 및 종료 조건 확인 ---
    
    # 이번 주기에 새 상품을 발견했다면, 실패 횟수 초기화
    if found_new_product_in_cycle:
        empty_scroll_count = 0
    # 새 상품을 발견하지 못했다면, 실패 횟수 1 증가
    else:
        empty_scroll_count += 1
        print(f"ℹ️ 이번 스크롤에서 새 상품을 찾지 못했습니다. (연속 실패: {empty_scroll_count}회)")

    # 연속 실패 횟수가 최대 허용치에 도달하면 루프 종료
    if empty_scroll_count >= max_empty_scrolls:
        print(f"\n{max_empty_scrolls}회 연속으로 새 상품이 없어 수집을 종료합니다.")
        break
        
    # 화면 높이만큼 부드럽게 스크롤
    # driver.execute_script("window.scrollBy(0, window.innerHeight);")
    # 로드된 페이지의 맨 아래로 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(1.5, 2.5))
    
    # 스크롤 후 새 콘텐츠가 로드될 시간을 보장 (WebDriverWait보다 이 방법이 더 안정적)
    time.sleep(random.uniform(1.5, 2.5))

print(f"\n✅ 1차 수집 완료: 총 {len(link_list)}개의 상품 링크를 찾았습니다.")

✅ [1] 수집 성공: 바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 1개
✅ [2] 수집 성공: 릴리이브 탈모케어 정수리 M자완화 볼륨 샴푸
✅ [3] 수집 성공: 바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 2개
✅ [4] 수집 성공: 러브밋 비건 저자극 탈모완화 볼륨 샴푸
✅ [5] 수집 성공: 라보에이치 탈모샴푸 약산성 대용량 두피 비건 400ml+400ml리필+180ml
✅ [6] 수집 성공: 려 루트젠 두피 에센스 대용량 145ml 두피 영양제 여성 남성 탈모 앰플
✅ [7] 수집 성공: 블루셀 마이크로바이옴 맥주효모 샴푸 알러젠프리향, 500ml, 1개
✅ [8] 수집 성공: 바이브랩 리바이 솔루션 안티 헤어 로스 스칼프 샴푸 우디플로럴머스크향, 500ml, 1개
✅ [9] 수집 성공: 려 루트젠 탈모샴푸 대용량 약산성 여성 비건 볼륨 두피 515ml+242ml+100ml
✅ [10] 수집 성공: 닥터포헤어 바이오3 폴리젠 탈모완화 샴푸 허브향, 500ml, 3개
✅ [11] 수집 성공: 네이처리퍼블릭 블랙빈 안티 헤어로스 샴푸(신형) 520ml, 2개
✅ [12] 수집 성공: 그래비티 엑스트라 스트롱 샴푸 475ml + 475ml(리필) + 30ml x 2 1개
✅ [13] 수집 성공: 메디큐브 소이민트 지루성두피 스케일링 비듬샴푸 490ml 2개
✅ [14] 수집 성공: 라보에이치 여름 쿨샴푸 지성 탈모 대용량 두피스케일링 400ml&400ml리필&180ml
✅ [15] 수집 성공: 반코르 맥주효모 탈모 완화 샴푸 프레쉬향, 500ml, 1개
✅ [16] 수집 성공: 1+1 살림백서 탈모 샴푸 엑티브B7 맥주효모 앤 비오틴 1000ml 남자 여자 바이오틴
✅ [17] 수집 성공: 메디큐브 소이민트 지루성두피 스케일링 비듬샴푸 490ml 2개
✅ [18] 수집 성공: 비건 탈모 샴푸 설페이트 계면활성제 없는 천연
✅ [19] 수집 성공: 그룬플러스 지성 쿨링 볼륨 탈모샴푸 250ml
✅ [20] 수집 

In [185]:
# 데이터 프레임 생성
raw = pd.DataFrame({
    '상품명':title_list, 
    '업체명':store_list, 
    '할인전가격':original_price_list, 
    '현재가격':current_price_list,
    '할인율':discount_rate_list,
    '링크':link_list,
    })
raw.shape

(1900, 6)

In [186]:
raw.head()

,상품명,업체명,할인전가격,현재가격,할인율,링크
0,"바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 1개",바이브랩,NaN,32000,NaN,https://ader.naver.com/v1/TkeeYEwH5_fC76zTuB6N...
1,릴리이브 탈모케어 정수리 M자완화 볼륨 샴푸,릴리이브,32000.0,19800,0.38,https://ader.naver.com/v1/dXtTlHvr9SsKUWqmzE59...
2,"바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 2개",바이브랩,NaN,62000,NaN,https://ader.naver.com/v1/wuNEwKNqZr79yqgQ_-xB...
3,러브밋 비건 저자극 탈모완화 볼륨 샴푸,러브밋 Luvmit,30000.0,24000,0.20,https://ader.naver.com/v1/rZudzJxAR0il3fWG8sPM...
4,라보에이치 탈모샴푸 약산성 대용량 두피 비건 400ml+400ml리필+180ml,아모레퍼시픽몰 헤어바디,44600.0,29800,0.33,https://smartstore.naver.com/main/products/900...


In [187]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상품명     1900 non-null   object 
 1   업체명     1900 non-null   object 
 2   할인전가격   1051 non-null   float64
 3   현재가격    1900 non-null   int64  
 4   할인율     1003 non-null   float64
 5   링크      1900 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 89.2+ KB


In [190]:
raw['할인전가격'].describe()

count      1051.00
mean      64230.19
std       78077.88
min         940.00
25%       30000.00
50%       45000.00
75%       70050.00
max     1190000.00
Name: 할인전가격, dtype: float64

In [191]:
raw.loc[0, '링크']

'https://ader.naver.com/v1/TkeeYEwH5_fC76zTuB6N_l52lbSahAVbwyguH92HcR1h-2k-R0PhxS-Ay-zn1r5RbUTbW-z0hbSvqGvgRKKd35hqEse1RyN0S9hsFJNz0qAGBobD_-dxrPt7R9T7N1vsXPaNnXQSSU0FOLErYlK7qa4zLlNIuCke7ark_KfW3mk4cPcCx4pqmRzKCoJVA5SXOSfphZNeB2BTuj66mfgI9TlQo1mdS-ZHjpZjG0SXOl3Mi5zwDIzM27LqCaCtn6M7skzVpswDJSyliuT1TB2gMzRxVxJw2GbLJatbXjwi7XPQ6UmCbE0l1MMNwV4TttU2le78dZ82fqvXlQ0XJfzLCW-UCnXMSeNkk_a3yr47mj-TY-mmKZKJgoqPibWcSeoeU7XuSLpu1gf8xpqow-Kv6HRqwSERm_VqFEJSSw4VX7Q94TN-MpqeNkbbIP5lIC22cnF53xSq6jjJKSGos7-SPKtnmwlUVk0j0SHbHWlqJHk=?c=pc.nplusstore.npla&t=0'

In [240]:
driver.get('https://brand.naver.com/lilyeve/products/12298697455?n_media=684927&n_query=%ED%83%88%EB%AA%A8&n_rank=2&n_ad_group=grp-a001-02-000000054331777&n_ad=nad-a001-02-000000414435525&n_campaign_type=2&n_mall_id=ncp_1oixnm_01&n_mall_pid=12298697455&n_ad_group_type=2&n_match=3&NaPm=ct%3Dmfd1jbki%7Cci%3DERd92a234e%2D8dc0%2D11f0%2D87a3%2Df2c9de48d820%7Ctr%3Dplan%7Chk%3D5903a84bcd48527a78ec9e761a169de1db71173f%7Cnacn%3DbAudBwwaYocW')
time.sleep(random.uniform(1, 2))
detail_html = driver.page_source
detail_soup = BeautifulSoup(detail_html, "html.parser")
print(detail_soup.title.text)

릴리이브 그로우턴 3D 볼륨 샴푸 500g 1개 : 릴리이브


In [241]:
# 수집할 정보의 키(key)를 리스트로 정의합니다.
labels_to_find = [
    "제조사", "브랜드", "모델명", "두피타입", "모발타입",
    "타입", "제품형태", "성분", "주요제품특징"
]

# 수집한 정보를 저장할 딕셔너리를 생성합니다.
product_info = {}

try:
    # 상품 정보가 담긴 모든 테이블의 헤더(<th>) 요소를 찾습니다.
    # 'rSg_SEReAx'는 정보 테이블에서 항목 이름이 있는 th 태그의 클래스명입니다.
    all_headers = driver.find_elements(By.CSS_SELECTOR, "table.RCLS1uAn0a th.rSg_SEReAx")

    # 찾은 모든 헤더를 순회하면서 원하는 정보를 찾습니다.
    for header in all_headers:
        header_text = header.text
        # 현재 헤더의 텍스트가 우리가 찾으려는 정보 리스트에 있는지 확인합니다.
        if header_text in labels_to_find:
            # 일치하는 헤더를 찾았다면, 바로 옆에 있는 데이터 셀(<td>)을 찾습니다.
            # XPath의 'following-sibling::td'는 같은 부모를 가진 다음 형제 td 요소를 의미합니다.
            value_cell = header.find_element(By.XPATH, "./following-sibling::td")
            
            # 딕셔너리에 '항목 이름: 값' 형태로 저장합니다.
            product_info[header_text] = value_cell.text

except Exception as e:
    print(f"정보를 수집하는 중 오류가 발생했습니다: {e}")

# 최종적으로 수집된 정보를 출력합니다.
print("--- 수집된 상품 정보 ---")
for key, value in product_info.items():
    print(f"{key}: {value}")
print("--------------------")

--- 수집된 상품 정보 ---
브랜드: 릴리이브
두피타입: 모든두피용, 지성, 건성
모발타입: 모든 모발용, 손상 모발용
타입: 탈모샴푸, 두피샴푸, 일반샴푸
제품형태: 펌프형
성분: 펩타이드
주요제품특징: 저자극, 세정력, 탈모케어
--------------------


In [248]:
# --- 2. 상세 정보 수집을 위한 리스트 및 변수 초기화 ---

# 상세 정보를 담을 리스트 초기화
manufacturer_list, brand_list, model_name_list = [], [], []
scalp_type_list, hair_type_list, type_list = [], [], []
product_form_list, ingredient_list, features_list = [], [], []

# --- 3. 상세 정보 1차 수집 ---
print(f"\n총 {len(link_list)}개의 상품 상세 정보 수집을 시작합니다.")

# link_list의 각 링크를 순회하며 상세 정보 수집
for i, link in enumerate(tqdm(link_list, desc="1차 상세 정보 수집 중")):
    try:
        # 해당 상품 상세 페이지로 이동
        driver.get(link)
        time.sleep(random.uniform(1.5, 2.5)) # 페이지 로딩 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h3.DCVBehA8ZB')))

        # 수집할 상세 정보 항목들
        labels_to_find = [
            "제조사", "브랜드", "모델명", "두피타입", "모발타입",
            "타입", "제품형태", "성분", "주요제품특징"
        ]
        
        # 현재 페이지의 정보를 임시로 담을 딕셔너리
        detail_info = {}

        # 상품 정보 테이블의 모든 항목(th)을 가져옴
        all_headers = driver.find_elements(By.CSS_SELECTOR, "table.RCLS1uAn0a th.rSg_SEReAx")

        for header in all_headers:
            header_text = header.text
            if header_text in labels_to_find:
                # 항목 이름(th) 바로 옆의 값(td)을 가져옴
                value_cell = header.find_element(By.XPATH, "./following-sibling::td")
                detail_info[header_text] = value_cell.text
        
        # 임시 딕셔너리에서 값을 가져와 각 리스트에 추가
        # .get(key, np.nan)을 사용하여 해당 정보가 없을 경우 NaN으로 처리
        manufacturer_list.append(detail_info.get("제조사", np.nan))
        brand_list.append(detail_info.get("브랜드", np.nan))
        model_name_list.append(detail_info.get("모델명", np.nan))
        scalp_type_list.append(detail_info.get("두피타입", np.nan))
        hair_type_list.append(detail_info.get("모발타입", np.nan))
        type_list.append(detail_info.get("타입", np.nan))
        product_form_list.append(detail_info.get("제품형태", np.nan))
        ingredient_list.append(detail_info.get("성분", np.nan))
        features_list.append(detail_info.get("주요제품특징", np.nan))

    except Exception as e:
        # 특정 페이지에서 오류 발생 시, 해당 상품의 모든 상세 정보는 NaN으로 채움
        print(f"⚠️ [{i+1}번째 링크] 처리 중 오류 발생: {link} ({e})")
        manufacturer_list.append(np.nan)
        brand_list.append(np.nan)
        model_name_list.append(np.nan)
        scalp_type_list.append(np.nan)
        hair_type_list.append(np.nan)
        type_list.append(np.nan)
        product_form_list.append(np.nan)
        ingredient_list.append(np.nan)
        features_list.append(np.nan)
        continue # 다음 링크로 넘어감

print("\n✅ 모든 상품의 상세 정보 수집을 완료했습니다.")


총 1900개의 상품 상세 정보 수집을 시작합니다.


1차 상세 정보 수집 중:   5%|▍         | 88/1900 [05:28<3:13:29,  6.41s/it]

⚠️ [88번째 링크] 처리 중 오류 발생: https://smartstore.naver.com/main/products/9610568744?nl-query=%ED%83%88%EB%AA%A8&nl-au=b50c5d1f778a4f5283e4a1202339f9ea&NaPm=ci%3Db50c5d1f778a4f5283e4a1202339f9ea%7Cct%3Dmfcmgvfm%7Ctr%3Dnslsl%7Csn%3D681720%7Chk%3D5bf5b77838091f39a346705ccfed4ad74d3c2e1f (Message: 
Stacktrace:
	GetHandleVerifier [0x0xaea8a3+63283]
	GetHandleVerifier [0x0xaea8e4+63348]
	(No symbol) [0x0x923e43]
	(No symbol) [0x0x96c8de]
	(No symbol) [0x0x96cc7b]
	(No symbol) [0x0x9b4ef2]
	(No symbol) [0x0x991464]
	(No symbol) [0x0x9b271a]
	(No symbol) [0x0x991216]
	(No symbol) [0x0x960855]
	(No symbol) [0x0x9616f4]
	GetHandleVerifier [0x0xd5bb43+2623955]
	GetHandleVerifier [0x0xd56daa+2604090]
	GetHandleVerifier [0x0xb1069a+218410]
	GetHandleVerifier [0x0xb00ed8+154984]
	GetHandleVerifier [0x0xb0742d+180925]
	GetHandleVerifier [0x0xaf22b8+94536]
	GetHandleVerifier [0x0xaf2442+94930]
	GetHandleVerifier [0x0xadd5ea+9338]
	BaseThreadInitThunk [0x0x769e5d49+25]
	RtlInitializeExceptionChain [0x0x77dd

1차 상세 정보 수집 중:   5%|▍         | 89/1900 [05:40<4:10:28,  8.30s/it]

⚠️ [89번째 링크] 처리 중 오류 발생: https://smartstore.naver.com/main/products/6790190553?nl-query=%ED%83%88%EB%AA%A8&nl-au=a20b732175ec47fb820434fb244cc2d5&NaPm=ci%3Da20b732175ec47fb820434fb244cc2d5%7Cct%3Dmfcmgvfn%7Ctr%3Dnslctg%7Csn%3D983877%7Chk%3D65dfaefa07a0ae32f4f9fd0e80853eaed25dd233 (Message: 
Stacktrace:
	GetHandleVerifier [0x0xaea8a3+63283]
	GetHandleVerifier [0x0xaea8e4+63348]
	(No symbol) [0x0x923e43]
	(No symbol) [0x0x96c8de]
	(No symbol) [0x0x96cc7b]
	(No symbol) [0x0x9b4ef2]
	(No symbol) [0x0x991464]
	(No symbol) [0x0x9b271a]
	(No symbol) [0x0x991216]
	(No symbol) [0x0x960855]
	(No symbol) [0x0x9616f4]
	GetHandleVerifier [0x0xd5bb43+2623955]
	GetHandleVerifier [0x0xd56daa+2604090]
	GetHandleVerifier [0x0xb1069a+218410]
	GetHandleVerifier [0x0xb00ed8+154984]
	GetHandleVerifier [0x0xb0742d+180925]
	GetHandleVerifier [0x0xaf22b8+94536]
	GetHandleVerifier [0x0xaf2442+94930]
	GetHandleVerifier [0x0xadd5ea+9338]
	BaseThreadInitThunk [0x0x769e5d49+25]
	RtlInitializeExceptionChain [0x0x77d

...

⚠️ [1453번째 링크] 처리 중 오류 발생: https://smartstore.naver.com/main/products/10280557719?nl-query=%ED%83%88%EB%AA%A8&nl-au=0ea94a4a0d6c4ee2b91ef4089995d440&NaPm=ci%3D0ea94a4a0d6c4ee2b91ef4089995d440%7Cct%3Dmfcmj38f%7Ctr%3Dnslsl%7Csn%3D4192588%7Chk%3D0be5c0356ad391d20ac6b606f1eb16c68905b4fc (Message: 
Stacktrace:
	GetHandleVerifier [0x0xaea8a3+63283]
	GetHandleVerifier [0x0xaea8e4+63348]
	(No symbol) [0x0x923e43]
	(No symbol) [0x0x96c8de]
	(No symbol) [0x0x96cc7b]
	(No symbol) [0x0x9b4ef2]
	(No symbol) [0x0x991464]
	(No symbol) [0x0x9b271a]
	(No symbol) [0x0x991216]
	(No symbol) [0x0x960855]
	(No symbol) [0x0x9616f4]
	GetHandleVerifier [0x0xd5bb43+2623955]
	GetHandleVerifier [0x0xd56daa+2604090]
	GetHandleVerifier [0x0xb1069a+218410]
	GetHandleVerifier [0x0xb00ed8+154984]
	GetHandleVerifier [0x0xb0742d+180925]
	GetHandleVerifier [0x0xaf22b8+94536]
	GetHandleVerifier [0x0xaf2442+94930]
	GetHandleVerifier [0x0xadd5ea+9338]
	BaseThreadInitThunk [0x0x769e5d49+25]
	RtlInitializeExceptionChain [0x0x

1차 상세 정보 수집 중:  77%|███████▋  | 1454/1900 [4:38:25<1:34:46, 12.75s/it]

⚠️ [1454번째 링크] 처리 중 오류 발생: https://smartstore.naver.com/main/products/11187645390?nl-query=%ED%83%88%EB%AA%A8&nl-au=ca967bafce4f4a82bd783f4fb7495680&NaPm=ci%3Dca967bafce4f4a82bd783f4fb7495680%7Cct%3Dmfcmj38f%7Ctr%3Dnslsl%7Csn%3D11874736%7Chk%3D073b203840e1e03db7b2310262ae2c8b497ad6a6 (Message: 
Stacktrace:
	GetHandleVerifier [0x0xaea8a3+63283]
	GetHandleVerifier [0x0xaea8e4+63348]
	(No symbol) [0x0x923e43]
	(No symbol) [0x0x96c8de]
	(No symbol) [0x0x96cc7b]
	(No symbol) [0x0x9b4ef2]
	(No symbol) [0x0x991464]
	(No symbol) [0x0x9b271a]
	(No symbol) [0x0x991216]
	(No symbol) [0x0x960855]
	(No symbol) [0x0x9616f4]
	GetHandleVerifier [0x0xd5bb43+2623955]
	GetHandleVerifier [0x0xd56daa+2604090]
	GetHandleVerifier [0x0xb1069a+218410]
	GetHandleVerifier [0x0xb00ed8+154984]
	GetHandleVerifier [0x0xb0742d+180925]
	GetHandleVerifier [0x0xaf22b8+94536]
	GetHandleVerifier [0x0xaf2442+94930]
	GetHandleVerifier [0x0xadd5ea+9338]
	BaseThreadInitThunk [0x0x769e5d49+25]
	RtlInitializeExceptionChain [0x0

1차 상세 정보 수집 중:  77%|███████▋  | 1454/1900 [4:38:31<1:25:26, 11.49s/it]


KeyboardInterrupt: 

In [ ]:
# --- 3. 데이터 프레임 생성 ---
# 기존에 수집한 리스트와 새로 수집한 상세 정보 리스트를 합쳐 데이터프레임 생성
raw = pd.DataFrame({
    '상품명': title_list,
    '판매처': store_list,
    '정상가': original_price_list,
    '할인가': current_price_list,
    '할인율': discount_rate_list,
    '제조사': manufacturer_list,
    '브랜드': brand_list,
    '모델명': model_name_list,
    '두피타입': scalp_type_list,
    '모발타입': hair_type_list,
    '타입': type_list,
    '제품형태': product_form_list,
    '성분': ingredient_list,
    '주요특징': features_list,
    '링크': link_list
})

In [233]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상품명     1900 non-null   object 
 1   판매처     1900 non-null   object 
 2   정상가     1051 non-null   float64
 3   할인가     1900 non-null   int64  
 4   할인율     1003 non-null   float64
 5   제조사     1510 non-null   object 
 6   브랜드     1520 non-null   object 
 7   모델명     897 non-null    object 
 8   두피타입    710 non-null    object 
 9   모발타입    699 non-null    object 
 10  타입      749 non-null    object 
 11  제품형태    670 non-null    object 
 12  주요특징    898 non-null    object 
 13  링크      1900 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 207.9+ KB


In [226]:
# --- 5. 자동 재시도 루프 ---

# 재시도 횟수 설정
MAX_RETRIES = 3

for i in range(MAX_RETRIES):
    # '브랜드' 정보가 비어있는(수집 실패한) 행을 찾습니다.
    failed_df = raw[raw['브랜드'].isnull()]
    
    # 만약 실패한 항목이 더 이상 없으면 재시도 루프를 종료합니다.
    if failed_df.empty:
        print("\n✅ 모든 항목 수집 완료. 재시도를 종료합니다.")
        break
        
    print(f"\n--- 재시도 {i+1}/{MAX_RETRIES} | 남은 항목: {len(failed_df)}개 ---")
    
    # 실패한 행들을 순회하며 재수집
    for index, row in tqdm(failed_df.iterrows(), total=len(failed_df), desc=f"재시도 {i+1}"):
        link = row['링크']
        try:
            driver.get(link)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'h3.DCVBehA8ZB'))
            )
            
            # (1차 수집과 동일한 정보 추출 로직)
            labels_to_find = [
                "제조사", "브랜드", "모델명", "두피타입", "모발타입",
                "타입", "제품형태", "성분", "주요제품특징"
            ]
            detail_info = {}
            all_headers = driver.find_elements(By.CSS_SELECTOR, "table.RCLS1uAn0a th.rSg_SEReAx")
            
            for header in all_headers:
                header_text = header.text
                if header_text in labels_to_find:
                    value_cell = header.find_element(By.XPATH, "./following-sibling::td")
                    detail_info[header_text] = value_cell.text

            # ⚠️ df.loc를 사용해 원본 데이터프레임의 해당 위치에 값을 '업데이트'합니다.
            raw.loc[index, '제조사'] = detail_info.get("제조사", np.nan)
            raw.loc[index, '브랜드'] = detail_info.get("브랜드", np.nan)
            raw.loc[index, '모델명'] = detail_info.get("모델명", np.nan)
            raw.loc[index, '두피타입'] = detail_info.get("두피타입", np.nan)
            raw.loc[index, '모발타입'] = detail_info.get("모발타입", np.nan)
            raw.loc[index, '타입'] = detail_info.get("타입", np.nan)
            raw.loc[index, '제품형태'] = detail_info.get("제품형태", np.nan)
            raw.loc[index, '성분'] = detail_info.get("성분", np.nan)
            raw.loc[index, '주요특징'] = detail_info.get("주요제품특징", np.nan)
            
        except Exception:
            # 재시도도 실패하면 그냥 넘어갑니다. (값은 여전히 NaN으로 유지)
            continue


--- 재시도 1/3 | 남은 항목: 427개 ---


재시도 1: 100%|██████████| 427/427 [16:36<00:00,  2.33s/it] 



--- 재시도 2/3 | 남은 항목: 381개 ---


재시도 2: 100%|██████████| 381/381 [14:40<00:00,  2.31s/it] 



--- 재시도 3/3 | 남은 항목: 380개 ---


재시도 3: 100%|██████████| 380/380 [14:52<00:00,  2.35s/it] 


In [227]:
# --- 6. 최종 결과 확인 ---
print("\n--- 최종 수집 결과 ---")
print(raw.info()) # 각 컬럼별 비어있지 않은 데이터 수를 확인


--- 최종 수집 결과 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상품명     1900 non-null   object 
 1   판매처     1900 non-null   object 
 2   정상가     1051 non-null   float64
 3   할인가     1900 non-null   int64  
 4   할인율     1003 non-null   float64
 5   제조사     1510 non-null   object 
 6   브랜드     1520 non-null   object 
 7   모델명     897 non-null    object 
 8   두피타입    710 non-null    object 
 9   모발타입    699 non-null    object 
 10  타입      749 non-null    object 
 11  제품형태    670 non-null    object 
 12  주요특징    898 non-null    object 
 13  링크      1900 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 207.9+ KB
None


In [247]:
df = raw.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900 entries, 0 to 1899
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상품명     1900 non-null   object 
 1   판매처     1900 non-null   object 
 2   정상가     1051 non-null   float64
 3   할인가     1900 non-null   int64  
 4   할인율     1003 non-null   float64
 5   제조사     1510 non-null   object 
 6   브랜드     1520 non-null   object 
 7   모델명     897 non-null    object 
 8   두피타입    710 non-null    object 
 9   모발타입    699 non-null    object 
 10  타입      749 non-null    object 
 11  제품형태    670 non-null    object 
 12  주요특징    898 non-null    object 
 13  링크      1900 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 207.9+ KB


In [249]:
df.to_csv('[processing]naver_shopping_hair_loss_20250910.csv', index=False)

In [252]:
# 드라이버 종료
driver.quit()